In [1]:
import pandas as pd

In [68]:
questions = pd.read_csv("data/questions.csv")
instructions = pd.read_csv("data/instructions.csv")

In [69]:
len(questions), len(instructions)

(8312, 8311)

In [73]:
questions.iloc[7598]["content"]

'Divide the letter "N" into halves.\n\n[STRDGM] The letter "N". [STPDGM]'

In [74]:
questions.head()

,subject,exercise_type,difficulty,grade_level,modifier,content
0,Computer Science,"Practical Exercise, Activity",D,2,M,Create a document file called 'My hobby' to us...
1,Computer Science,Algorithm Writing Exercise,D,2,Exercise on making a toy horse,Write an algorithm for making a toy horse. \nW...
2,Computer Science,Short Answer Exercise,D,2,With Material,What does ‘algorithm' mean? Write or say the a...
3,Computer Science,Short Answer Exercise,D,5,With Material,Imagine that the x coordinate of an apple is a...
4,Computer Science,Programming Exercise,D,5,"With Material, Exercise on sprite creation and...","Add a sprite (for example, an apple) to your s..."


In [72]:
for i in range(len(instructions)):
    q_exercise_types = questions.iloc[i]["exercise_type"].split(",")
    # print(q_exercise_types)
    [t.lower() in instructions.iloc[i]["Input"].lower() for t in q_exercise_types]
    if all(t.lower() in instructions.iloc[i]["Input"].lower() for t in q_exercise_types):
        # print(True, i)
        pass
    else:
        print(False, i)
    # break

False 7598
False 7600
False 7601
False 7602
False 7604
False 7606
False 7607
False 7608
False 7610
False 7613
False 7614
False 7616
False 7617
False 7618
False 7621
False 7623
False 7624
False 7627
False 7629
False 7632
False 7633
False 7634
False 7647
False 7648
False 7650
False 7653
False 7656
False 7657
False 7658
False 7659
False 7664
False 7666
False 7667
False 7671
False 7672
False 7673
False 7677
False 7680
False 7683
False 7684
False 7687
False 7691
False 7692
False 7696
False 7700
False 7701
False 7705
False 7709
False 7710
False 7711
False 7718
False 7720
False 7724
False 7729
False 7733
False 7734
False 7736
False 7739
False 7741
False 7742
False 7743
False 7745
False 7747
False 7748
False 7749
False 7751
False 7756
False 7761
False 7769
False 7770
False 7771
False 7772
False 7773
False 7774
False 7776
False 7785
False 7786
False 7787
False 7788
False 7791
False 7792
False 7793
False 7794
False 7796
False 7798
False 7800
False 7801
False 7802
False 7803
False 7806
False 7807

I have a dataframe with 5 columns: "subject"	"exercise_type"	"difficulty"	"grade_level"	"modifier"	"content".
Now I want to randomly sample 2 rows and calculate the number of matching columns ("content" is exlcluded), for instance, row1["subject"] == row2["subject"] --> 1 matching
Create a dataframe to store sample results, each result contains "num_matched_column", "row1.index", "row2.index"

In [83]:
import pandas as pd
import numpy as np

# Define function to sample and calculate matches
def sample_and_match(df, sample_size=2):
    # Sample 2 rows
    sampled_rows = df.sample(sample_size)
    
    # Extract the indices of the sampled rows
    idx1, idx2 = sampled_rows.index
    
    # Exclude "content" and "difficulty" column from the comparison
    cols_to_compare = df.columns.difference(['content', 'difficulty'])
    
    # Count the number of matching columns (excluding "content" and "difficulty")
    num_matched_columns = np.sum(sampled_rows.iloc[0][cols_to_compare] == sampled_rows.iloc[1][cols_to_compare])
    
    # Create a result dictionary
    result = {
        'num_matched_column': num_matched_columns,
        'row1.index': idx1,
        'row2.index': idx2
    }
    
    return result

# Example usage
result_df = sample_and_match(questions)
print(result_df)

{'num_matched_column': 0, 'row1.index': 5305, 'row2.index': 3750}


In [84]:
questions.iloc[8097], questions.iloc[5017]

(subject                                                Mathematics
 exercise_type                       Algebra Exercise, Word Problem
 difficulty                                                       D
 grade_level                                                      6
 modifier           Exercise with algebraic expressions, Multi-part
 content          Chih-chieh spent \$13 and now has \( c \) doll...
 Name: 8097, dtype: object,
 subject                                                Mathematics
 exercise_type                                         Word problem
 difficulty                                                       D
 grade_level                                                      6
 modifier                               Exercise that uses a recipe
 content          Chocolate ice cream for four people uses vanil...
 Name: 5017, dtype: object)

In [89]:
# Create 100 samples and store results in a DataFrame
results = []
for _ in range(10000):
    result = sample_and_match(questions)
    results.append(result)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Display the resulting DataFrame
# print(results_df)

# Display the distribution of different values of num_matched_column
distribution = results_df['num_matched_column'].value_counts().sort_index()

# Display the distribution
print(distribution)

num_matched_column
0    6622
1    2714
2     601
3      62
4       1
Name: count, dtype: int64


### Parse and build questions dataframe with param columns

In [64]:
import re
import pandas as pd

def parse_params(text):
    text = text.strip()
    # Find content '(...)' or S or T or D or G or M or number (5 params)
    params= re.findall(r"\(.+?\)|\d+|S|T|D|G|M", text)
    params = [p.replace("(","").replace(")", "") for p in params]
    # assert len(params) == 5, "Parse result is not complete"
    assert len(params) == 5, text
    return params

with open("data/questions.txt", "r") as f:
    text = f.read()

text = text
title_pattern = r"\([a-zA-Z].+\).+ D .+\n"
# Find all titles (questions)
titles = re.findall(title_pattern, text)
# Find all contents (answers)
contents = re.split(title_pattern, text)
contents = [content.strip() for content in contents]
contents = contents[1:] if contents[0] == "" else contents

In [67]:
subject, exercise_type, difficulty, grade_level, modifier = ([] for _ in range(5))

for title in titles:
    params = parse_params(title)
    subject.append(params[0])
    exercise_type.append(params[1])
    difficulty.append(params[2])
    grade_level.append(params[3])
    modifier.append(params[4])

struct_data = pd.DataFrame({"subject": subject, "exercise_type": exercise_type, "difficulty": difficulty,
                             "grade_level": grade_level, "modifier": modifier, "content": contents})

struct_data.to_csv("data/questions.csv",index=False)